# 😼 Практическая часть лекции "Использование готовых предобученных моделей для создания ML-driven продуктов"

## План ноутбука:
1. Hard-способ: дообучение модели детекции через PyTorch.
2. Light-способы: YOLO, RoboFlow, Hugging Face.


## Hard-способ
Находить предобученные веса моделей в библиотеках PyTorch или TensorFlow: конкретную/лучшую архитектуру можно найти на [paperswithcode.com](https://paperswithcode.com/) ~или по запросу в Google - например, "SOTA image classification models"~.

Смотрим ТОП по метрикам, выбираем модель - как правило, все инструкции и веса лежат на GitHub - и дообучаем на своих данных.

💥 Давайте попробуем решить задачу классификации - распознавание людей на фотографии: наша модель будет уметь распознавать Маска, Гейтса, Безоса, Цукерберга и Джобса. Соответствующие датасеты мы уже собрали:)

In [1]:
import os
import torch
import random
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import shutil
import time
import xml.etree.ElementTree as et

from tqdm import tqdm
from PIL import Image
from torchvision import models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
BATCH_SIZE = 32

In [2]:
use_gpu = torch.cuda.is_available()
device = 'cuda' if use_gpu else 'cpu'
print('Connected device:', device)

musk = 'https://drive.google.com/uc?export=download&id=1BOuq35QzO1YtKQkYGfj_vtBj3Ps5xyBN'
gates ='https://drive.google.com/uc?export=download&id=1jgHQF_NMpH9uMTvic9rGnURu_8UOGdiz'
bezos = 'https://drive.google.com/uc?export=download&id=1n5UaLL-TAkjIeBbTNcn-Czkp_A3Eslhj'
zuker = 'https://drive.google.com/uc?export=download&id=1ncPmYTg6EPHlUFdcjl_bXTbtWRLv2DXy'
jobs = 'https://drive.google.com/uc?export=download&id=1TX3hiRyvSYiYVZUFrbAhN3Jpp9cd0Q9s'

face_lst=[
    ["Elon Musk",'data/musk500.jpg'],
    ["Bill Gates",'data/gates500.jpg'],
    ["Jeff Besoz",'data/bezos500.jpg'],
    ["Mark Zuckerberg", 'data/zuckerberg500.jpg'],
    ["Steve Jobs",'data/jobs500.jpg']
]

Connected device: cuda


In [3]:
 ! pwd

/home/gna/workspase/education/MEPHI/MEPHI+SF_ML/academic-semester-1/hackathon


In [4]:
import wget
from zipfile import ZipFile

os.mkdir('data')

url = 'https://drive.google.com/uc?export=download&id=120xqh0mYtYZ1Qh7vr-XFzjPbSKivLJjA'
file_name = wget.download(url, 'data/')

with ZipFile(file_name, 'r') as zip_file:
    zip_file.extractall()

link_lst = [musk, gates, bezos, zuker, jobs]
for link in link_lst:
    wget.download(link, 'data/')

FileExistsError: [Errno 17] File exists: 'data'

In [5]:
# Training dataset.
train_dataset = ImageFolder(
    root='data/train'
)
# Validation dataset.
valid_dataset = ImageFolder(
    root='data/valid'
)

Добавим аугментаций - автовыравнивание изображений и автоконтраста (чтобы улучшить обобщающую способность моделей и внизить риск переобучения).

In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
train_dataset.transform = transforms.Compose([
    transforms.Resize([70, 70]),
    transforms.RandomHorizontalFlip(), # augmentations
    transforms.RandomAutocontrast(), # augmentations
    transforms.RandomEqualize(), # augmentations
    transforms.ToTensor(),
    normalize
])

valid_dataset.transform = transforms.Compose([
    transforms.Resize([70, 70]),
    transforms.ToTensor(),
    normalize
])


In [7]:
# Training data loaders.
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE,
    shuffle=True
)
# Validation data loaders.
valid_loader = DataLoader(
    valid_dataset, batch_size=BATCH_SIZE,
    shuffle=False
)

Возьмём предобученную нейросеть GoogLeNet.

Меняем выходной параметр `Linear(out_feature=1000)` на количество классов.

Замораживаем все исходные слои модели.

In [9]:
def google(): # pretrained=True для tensorflow
    model = models.googlenet(weights=models.GoogLeNet_Weights.IMAGENET1K_V1)
    model.fc = torch.nn.Linear(1024, len(train_dataset.classes))
    for param in model.parameters():
        param.requires_grad = True
    model.inception3a.requires_grad = False
    model.inception3b.requires_grad = False
    model.inception4a.requires_grad = False
    model.inception4b.requires_grad = False
    model.inception4c.requires_grad = False
    model.inception4d.requires_grad = False
    model.inception4e.requires_grad = False
    return model

In [10]:
model = google()
model

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

Аналогичный пример для предобученной сети VGG19.

In [ ]:
def vgg_19():
    model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
    model.classifier[6] = torch.nn.Linear(4096, len(train_dataset.classes))
    for param in model.features.parameters():
        param.requires_grad = False
    for index, block in enumerate(model.features):
        if index >= 30:
            for param in block.parameters():
                param.requires_grad = True
    return model

Определяем функцию обучения модели (это ± стандартная вещь).

In [11]:
def train(model, optimizer, train_loader, val_loader, epoch=10):
    loss_train, acc_train = [], []
    loss_valid, acc_valid = [], []
    for epoch in tqdm(range(epoch)):
        losses, equals = [], []
        torch.set_grad_enabled(True)

        # Train.
        model.train()
        for i, (image, target) in enumerate(train_loader):
            image = image.to(device)
            target = target.to(device)
            output = model(image)
            loss = criterion(output,target)

            losses.append(loss.item())
            equals.extend(
                [x.item() for x in torch.argmax(output, 1) == target])

            optimizer.zero_grad() 
            loss.backward() # движение назад
            optimizer.step() # движение назад
        loss_train.append(np.mean(losses))
        acc_train.append(np.mean(equals))
        losses, equals = [], []
        torch.set_grad_enabled(False)

        # Validate.
        model.eval()
        for i , (image, target) in enumerate(valid_loader):
            image = image.to(device)
            target = target.to(device)

            output = model(image)
            loss = criterion(output,target)

            losses.append(loss.item())
            equals.extend(
                [y.item() for y in torch.argmax(output, 1) == target])

        loss_valid.append(np.mean(losses))
        acc_valid.append(np.mean(equals))

    return loss_train, acc_train, loss_valid, acc_valid

Дальше код для ~до~обучения модели.

In [12]:
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)

model = google() # здесь можете заменить на VGG
print('Model: GoogLeNet\n')

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model = model.to(device)

loss_train, acc_train, loss_valid, acc_valid = train(
model, optimizer, train_loader, valid_loader, 30)
print('acc_train:', acc_train, '\nacc_valid:', acc_valid)
del model
torch.cuda.empty_cache()

Model: GoogLeNet



  0%|          | 0/30 [00:00<?, ?it/s]/home/gna/anaconda3/envs/rocm/lib/python3.10/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at ../aten/src/ATen/Context.cpp:296.)
  return F.linear(input, self.weight, self.bias)
100%|██████████| 30/30 [01:36<00:00,  3.21s/it]

acc_train: [0.672, 0.915, 0.9593333333333334, 0.974, 0.983, 0.9866666666666667, 0.9863333333333333, 0.9906666666666667, 0.992, 0.9946666666666667, 0.9946666666666667, 0.9953333333333333, 0.9936666666666667, 0.9966666666666667, 0.9973333333333333, 0.9956666666666667, 0.9946666666666667, 0.998, 0.996, 0.9966666666666667, 0.9966666666666667, 0.9976666666666667, 0.998, 0.999, 0.999, 0.998, 0.9966666666666667, 0.9983333333333333, 0.995, 0.9956666666666667] 
acc_valid: [0.8894967177242888, 0.9606126914660832, 0.975929978118162, 0.9781181619256017, 0.9846827133479212, 0.9879649890590809, 0.9923413566739606, 0.9923413566739606, 0.9923413566739606, 0.9923413566739606, 0.9890590809628009, 0.9901531728665208, 0.9923413566739606, 0.9901531728665208, 0.9934354485776805, 0.9934354485776805, 0.9923413566739606, 0.9901531728665208, 0.9934354485776805, 0.9923413566739606, 0.9923413566739606, 0.9912472647702407, 0.9945295404814004, 0.9934354485776805, 0.9912472647702407, 0.9956236323851203, 0.9934354485

: 

## Light-способы
💥 **1.** Для некоторых популярных моделей, оформленных в отдельные библиотеки, функции дообучения реализованы максимально удобно: вплоть до того, что достаточно вызвать их в несколько строк.

Для задачи детекции в CV рекомендуем рассмотреть модель ***YOLO***, те самые "несколько строчк" можно найти [тут](https://github.com/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb). Да и в принципе в [официальном репозитории](https://github.com/ultralytics/ultralytics) много всего интересного:)

💥 **2.** Хороший сервис-хранилище предобученных моделей - [***RoboFlow***](https://universe.roboflow.com/), в нём можно найти множество моделей для различных задач и легко использовать их по API. Сервис специализируется в основном на CV-задачах.

**Алгоритм работы c RoboFlow:**

1. Зарегистрироваться (можно с аккаунта google)
2. Самое доступное, чем можно воспользоваться - найти готовый проект через [страницу поиска](https://universe.roboflow.com/search?q=nlp) и запустить его "инференс" (к примеру, например, [вот](https://universe.roboflow.com/alphabettraining/character-detection-iis85/model/2) модель детекции цифр на фото  - внизу находите Hosted API и код как легко запустить модель у себя.
3. При неточном совпадении задачи может потребоваться *дообучить модель*. По нужным темам тоже много туториалов, например, про дообучение через веб-интерфейс Roboflow можно посмотреть в [туториале про YOLO](https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset/)

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="API_KEY")
project = rf.workspace().project("deneme3-pc88k")
model = project.version(8).model

# infer on a local image
print(model.predict("your_image.jpg", confidence=40, overlap=30).json())

# visualize your prediction
# model.predict("your_image.jpg", confidence=40, overlap=30).save("prediction.jpg")

# infer on an image hosted elsewhere
# print(model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json())

💥 **3.** И крупное хранилище различных моделей - [Hugging Face](https://huggingface.co/), там можно найти всё по NLP и по CV:)

Ссылка на примеры с NLP [тут](https://colab.research.google.com/drive/1C-OEAr2Kk2NYBlb-dfVjfV1P4bxjh4Tp).

**Ещё полезные ссылки:**

pickle, onnx - библиотеки для сохранения моделей
pickle - load, dump

Использование chatGPT через API: https://ihsavru.medium.com/how-to-build-your-own-custom-chatgpt-using-python-openai-78e470d1540e

Получение API: https://platform.openai.com/api-keys

Как сделать сервер на Flask https://youtu.be/MxJnR1DMmsY